In [1]:
import bs4
import urllib
import re
import pandas as pd
import time

In [2]:
query = 'http://www.oferty.net/mieszkania/szukaj?ps[type]=1&ps[location][type]=1&ps[location][text_queue][]=Wroc%C5%82aw&ps[transaction]=1&ps[date_filter]=0&ps[sort_order]=rank_asc'

class_filter = lambda x: x and 'property' in x.split()
attr_defs = [('cell_rooms', int), 
             ('cell_area', lambda x: float(re.match('([0-9,]+) m.*', x).groups()[0].replace(',','.'))), 
             ('cell_price', lambda x: float(re.match('([0-9 ]+) PLN', x).groups()[0].replace(' ',''))/1e3)
            ]

scraped = []

for page_num in xrange(350):
    if (page_num % 50)==0:
        print 'On page: %d' %(page_num)
    
    sock = urllib.urlopen('%s&page=%d' % (query ,page_num+1))
    html_data = sock.read()
    sock.close()
    
    soup = bs4.BeautifulSoup(html_data)
    
    for data_row in soup.find_all('tr', class_=class_filter):
        attrs = []
        try:
            for attr_name, attr_conv in attr_defs:
                attr_data = data_row.find("td", class_=attr_name).text.strip() 
                attrs.append(attr_conv(attr_data))
            scraped.append(attrs)
        except:
            pass
    
    time.sleep(1)
        
house_prices = pd.DataFrame(scraped, columns=[n[5:] for (n,c) in attr_defs])

On page: 0


/pio/os/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


On page: 50
On page: 100
On page: 150
On page: 200
On page: 250
On page: 300


In [3]:
house_prices.to_csv('03-house-prices.csv')